In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from scipy.spatial import distance
from scipy.sparse import csr_matrix
from numpy import transpose

In [2]:
def TSCAN(path, X):
    v12 = np.array(path[1]).reshape(-1,1) -np.array(path[0]).reshape(-1,1)
    project_value_list1 = []
    for item in X[0]:
        x1 = np.array(item).reshape(-1,1)  - np.array(path[0]).reshape(-1,1)
        project_value1 = (x1.T@(v12))/(v12.T@v12) 
        project_value_list1.append(project_value1)

    
    v_last = np.array(path[-1]).reshape(-1,1)-np.array(path[-2]).reshape(-1,1)
    project_value_list_last = []
    for item in X[-1]:
        x_last = np.array(item).reshape(-1,1)  - np.array(path[-1]).reshape(-1,1)
        project_value_last = (x_last.T@(v_last))/(v_last.T@v_last) 
        project_value_list_last.append(project_value_last)

    lists = []
    for i in range(1, len(path)-1):
        project_value_list_minus = []
        project_value_list_add = []

        v_minus = np.array(path[i]).reshape(-1,1)-np.array(path[i-1]).reshape(-1,1)
        v_add = np.array(path[i+1]).reshape(-1,1)-np.array(path[i]).reshape(-1,1)
        for item in X[i]:
            d1 = np.linalg.norm(np.array(item) - path[i-1])
            d2 = np.linalg.norm(np.array(item) - path[i+1])
            xi = np.array(item).reshape(-1,1) - np.array(path[i]).reshape(-1,1)
            if d1 < d2:
                project_value_minus = (xi.T@(v_minus))/(v_minus.T@v_minus)
                project_value_list_minus.append(project_value_minus)
                
            else:
                project_value_add = (xi.T@(v_add))/(v_add.T@v_add)
                project_value_list_add.append(project_value_add)
                
            combine = (project_value_list_minus, project_value_list_add)
        lists.append(combine)
        flat_lists = list(np.concatenate(lists).flat)
        remove_empty = [x for x in flat_lists if x]
        project_value_int = list(np.concatenate(remove_empty).flat)
        

        
    order_c1 = np.argsort(np.array(project_value_list1).reshape(1, -1))
    order_c1_list = order_c1[0].tolist()
    
    cell_order_int = np.argsort(np.array(project_value_int, dtype=object).reshape(1, -1))
    order_int = cell_order_int[0] + max(order_c1[0])
    order_int_list = order_int.tolist()
    
    
    cell_order_last = np.argsort(np.array(project_value_list_last,dtype=object).reshape(1, -1))
    order_last = cell_order_last[0]+max(order_int)
    order_last_list = order_last.tolist()
    
    
    
    orders_combine = order_c1_list + order_int_list + order_last_list
    orders = np.transpose(orders_combine)
        
    return orders
        


In [3]:
data = pd.read_csv("matrixnby2.csv")
matrix = np.array(data)
kmeans = KMeans(n_clusters=4)
kmeans.fit(matrix)
y_kmeans = kmeans.predict(matrix)

In [4]:
path = kmeans.cluster_centers_
path

array([[399.11798842,  91.52706719],
       [193.65413298,  65.39694807],
       [222.27945767, 174.99617564],
       [445.17824832,  16.99463969]])

In [5]:
labels = kmeans.labels_
labels

array([1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1,
       2, 2, 1, 0, 3, 0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 0, 3, 3, 3, 0, 0, 3,
       0, 0, 0, 1, 3, 0, 0, 3, 0, 0, 3], dtype=int32)

In [6]:
c1_points_position = list(np.where(labels == 0)[0])
c2_points_position = list(np.where(labels == 1)[0])
c3_points_position = list(np.where(labels == 2)[0])
c4_points_position = list(np.where(labels == 3)[0])


x1 = [matrix[ele] for ele in c1_points_position ]
x2 = [matrix[ele] for ele in c2_points_position ]
x3 = [matrix[ele] for ele in c3_points_position ]
x4 = [matrix[ele] for ele in c4_points_position ]


In [7]:
X=(x1,x2,x3,x4)


In [9]:
TSCAN(path, X)

array([ 2, 10,  0, 15,  4,  3, 12,  9,  8,  7, 17, 11, 16,  6, 14,  1, 13,
        5, 22, 21, 36, 42, 20, 17, 41, 29, 18, 39, 35, 34, 27, 37, 40, 23,
       30, 31, 28, 24, 38, 25, 32, 26, 19, 33, 47, 42, 50, 46, 45, 43, 48,
       44, 52, 49, 51])